In [3]:
pip install folium

     -------------------------------------- 102.3/102.3 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from tqdm import trange
from tqdm import tqdm
import folium
import time

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

service = Service()
driver = webdriver.Chrome(service=service)

# 윈도우 화면 최대화
driver.maximize_window()

content_link = []
title = []
writer = []
rank = []
df = pd.DataFrame()
days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'finish']

URL = f'https://comic.naver.com/webtoon'
driver.get(URL)
time.sleep(15)
    
for day in days:
    URL = f'https://comic.naver.com/webtoon?tab={day}'
    driver.get(URL)
    
    prior = 0
    new_height = 0
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 1초 대기
        time.sleep(3)

        prior = new_height
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 더이상 늘어나지 않으면 반복문 중단
        if new_height == prior:
            break
    
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    container = soup.find("ul", class_="ContentList__content_list--q5KXY")
    
    tbody = container.find_all("li")
    print(f'{day} 개수:', len(tbody)) # 1824

    for i in tbody:
            c = i.find("a",class_="Poster__link--sopnC")
            content_link.append(c['href'])
            
            t = i.find('span', class_='ContentTitle__title--e3qXt')
            t2 = t.find('span', class_='text').text
            title.append(t2)
            
            w = i.find('a', class_='ContentAuthor__author--CTAAP').text
            writer.append(w)
            
            r = i.find('span', class_='Rating__star_area--dFzsb')
            r2 = r.find('span', class_='text').text
            rank.append(r2)
      
    time.sleep(1)        

df['내용링크'] = content_link
df['제목'] = title
df['작가'] = writer
df['별점'] = rank

mon 개수: 93
tue 개수: 93
wed 개수: 92
thu 개수: 92
fri 개수: 90
sat 개수: 93
sun 개수: 89
finish 개수: 1828


In [5]:
df.head()

,내용링크,제목,작가,별점
0,/webtoon/list?titleId=758037,참교육,채용택 / 한가람,9.87
1,/webtoon/list?titleId=648419,뷰티풀 군바리,설이 / 윤성원,9.81
2,/webtoon/list?titleId=183559,신의 탑,SIU,9.88
3,/webtoon/list?titleId=733074,백수세끼,치즈,9.85
4,/webtoon/list?titleId=774866,똑 닮은 딸,이담,9.98


In [6]:
df.tail()

,내용링크,제목,작가,별점
2465,/webtoon/list?titleId=20762,AA,"선정, 지숙 / 지숙, 선정",9.68
2466,/webtoon/list?titleId=15439,사랑in,전세훈,9.51
2467,/webtoon/list?titleId=22073,까뱅,X-TEAM,8.84
2468,/webtoon/list?titleId=15938,와탕카,우주인,9.63
2469,/webtoon/list?titleId=15569,타투,노지현,9.20


In [9]:
df.shape

(2470, 4)

In [10]:
df.to_csv('naver_content_link.csv', index=False)

In [11]:
plot = []
tags = []
heart = []
age = []
total = []
image = []

In [25]:
service = Service()
driver = webdriver.Chrome(service=service)

for i in df['내용링크']:
    try:
        url = 'https://comic.naver.com' + i
        driver.get(url)
        time.sleep(1)

        plot.append(driver.find_element(By.CSS_SELECTOR, 'p.EpisodeListInfo__summary--Jd1WG').text)
        tags.append(driver.find_element(By.CSS_SELECTOR, 'div.TagGroup__tag_gro|up--uUJza').text.replace('\n',''))
        age.append(driver.find_element(By.CSS_SELECTOR, '#content > div.EpisodeListInfo__comic_info--yRAu0 > div > div.ContentMetaInfo__meta_info--GbTg4 > em').text.split('∙')[-1].replace('\n',''))
        total.append(driver.find_element(By.CSS_SELECTOR, '#content > div.EpisodeListView__episode_list_wrap--q0VYg > div.EpisodeListView__episode_list_head--PapRv > div.EpisodeListView__count--fTMc5').text)
        heart.append(driver.find_element(By.CSS_SELECTOR, '#content > div.EpisodeListView__user_wrap--S_pYn > div > button.EpisodeListUser__item--Fjp4R.EpisodeListUser__favorite--DzoPt > span.EpisodeListUser__count--fNEWK').text.replace(',',''))
        image.append(driver.find_element(By.CSS_SELECTOR, '#content > div.EpisodeListView__episode_list_wrap--q0VYg > ul > li:nth-child(1) > a > div.EpisodeListList__thumbnail_area--EL1aw > img').get_attribute('src'))
            
    except:
        plot.append('연령확인필요')
        tags.append('연령확인필요')
        age.append('연령확인필요')
        total.append('연령확인필요')
        heart.append('연령확인필요')
        image.append('연령확인필요')

In [23]:
len(tags)

17291

In [24]:
df['줄거리'] = plot
df['태그'] = tags
df['관심'] = heart
df['이용가'] = age
df['몇화'] = total
df['이미지링크'] = image

ValueError: Length of values (17291) does not match length of index (2470)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
df.to_csv('naver_webtoon.csv', index=False)